In [1]:
!pip install yfinance pandas ta


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd

In [3]:
ticker = "NVDA"
data = yf.download(ticker, period="max", interval="1d")  # 1 year of daily data

Failed to get ticker 'NVDA' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)')))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [4]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,


In [5]:
# Simple Moving Averages
data['MA10'] = data['Close'].rolling(window=10).mean()  # 10-day MA
data['MA50'] = data['Close'].rolling(window=50).mean()  # 50-day MA

# Exponential Moving Averages
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()  # 10-day EMA
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()  # 50-day EMA

In [6]:
from ta.momentum import RSIIndicator

# Add RSI (default is 14-day period)
data['RSI'] = RSIIndicator(data['Close']['NVDA']).rsi()

In [7]:
from ta.volatility import BollingerBands

# Calculate Bollinger Bands
bb_indicator = BollingerBands(data['Close']['NVDA'])
data['BB_upper'] = bb_indicator.bollinger_hband()
data['BB_lower'] = bb_indicator.bollinger_lband()
data['BB_middle'] = bb_indicator.bollinger_mavg()

In [8]:
from ta.volatility import AverageTrueRange

# Calculate ATR
data['ATR'] = AverageTrueRange(data['High']['NVDA'], data['Low']['NVDA'], data['Close']['NVDA']).average_true_range()

IndexError: index 13 is out of bounds for axis 0 with size 0

In [ ]:
# Flatten MultiIndex columns
data.columns = ['_'.join(col).strip() for col in data.columns.values]

In [ ]:
data

In [ ]:
tickers = ["NVDA", "AMD", "INTC", "^GSPC", "^IXIC"]

# Fetch data for each ticker
data_dict = {}
for ticker in tickers:
    data_dict[ticker] = yf.download(ticker, period="max", interval="1d")

In [ ]:
# Extract relevant columns and rename them
for ticker in tickers:
    data[f"{ticker}_Close"] = data_dict[ticker]['Close']

In [ ]:
from dotenv import load_dotenv
import os
import requests

# Load environment variables from .env file
load_dotenv()

# Access the API key
FRED_API_KEY = os.getenv("FRED_API_KEY")

In [ ]:
interest_endpoint = f"https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key={FRED_API_KEY}&file_type=json"
interest_response = requests.get(interest_endpoint)

In [ ]:
gdp_endpoint = f"https://api.stlouisfed.org/fred/series/observations?series_id=GDP&api_key={FRED_API_KEY}&file_type=json"
gdp_response = requests.get(gdp_endpoint)

In [ ]:
start_date = data.index.min().strftime('%Y-%m-%d')
end_date = data.index.max().strftime('%Y-%m-%d')

print("Start Date:", start_date)
print("End Date:", end_date)

In [ ]:
gdp_data = gdp_response.json()

In [ ]:
gdp_df = pd.DataFrame(gdp_data['observations'])
gdp_df['date'] = pd.to_datetime(gdp_df['date'])
gdp_df['value'] = pd.to_numeric(gdp_df['value'], errors='coerce')
gdp_df = gdp_df.rename(columns={'date': 'Date', 'value': 'GDP'}).set_index('Date')

In [ ]:
gdp_daily = gdp_df.resample('D').ffill()

In [ ]:
gdp_daily = gdp_daily[start_date:end_date]

In [ ]:
gdp_daily

In [ ]:
interest_data = interest_response.json()

In [ ]:
interest_df = pd.DataFrame(interest_data['observations'])
interest_df['date'] = pd.to_datetime(interest_df['date'])
interest_df['value'] = pd.to_numeric(interest_df['value'], errors='coerce')
interest_df = interest_df.rename(columns={'date': 'Date', 'value': 'Interest_Rate'}).set_index('Date')

In [ ]:
interest_daily = interest_df.resample('D').ffill()

In [ ]:
interest_daily = interest_daily[start_date:end_date]

In [ ]:
interest_daily

In [ ]:
# Merge GDP and Interest Rate into data
data = data.merge(gdp_daily, how='left', left_index=True, right_index=True)
data = data.merge(interest_daily, how='left', left_index=True, right_index=True)

In [ ]:
data

In [ ]:
data = data.drop(columns=['realtime_start_x', 'realtime_end_x', 'realtime_start_y', 'realtime_end_y'], errors='ignore')

In [ ]:
data.to_csv('uncleaned.csv')

In [ ]:
data